In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time
import os



c:\Users\Jose\anaconda3\envs\ProyectoNegocios\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class RecommenderEngine:
    # Motor de recomendación
    def __init__(self, model_name, embedding_path, index_path):
        
        
        # 1: Validación de arcgivos
        if not os.path.exists(embedding_path):
            raise FileNotFoundError(f"No se encontró el archivo con los embeddings {embedding_path}")
        if not os.path.exists(index_path):
            raise FileNotFoundError(f"No se encontró el archivo de índice {index_path}")
                                    
        # 2: Cargar modelo BERT
        self.model = SentenceTransformer(model_name)
        
        # 3: Cargar embeddings
        self.perfume_embeddings = np.load(embedding_path)
        
        # 4: Cargar índice de perfumes
        self.perfume_index = pd.read_csv(index_path)
        
        
        print(f"Total perfumes cargados: {len(self.perfume_index)}")
        print(f"Forma matriz embedida: {self.perfume_embeddings.shape}")
    

    def find_similar(self, query_text, top_n=5):
        # Encuentra el top n perfumes similares dada la consulta.
        start_time = time.time()
        
        # 1: Codificar la consulta
        query_embedding = self.model.encode([query_text])
        
        # 2: Calcular la similitud del coseno
        cosine_sims = cosine_similarity(query_embedding, self.perfume_embeddings).flatten()
        
        # 3: Obtener índices del top n
        top_indices = cosine_sims.argsort()[-top_n:][::-1]
        
        # 4: Preparar resultados
        results_df = self.perfume_index.iloc[top_indices].copy()
        
        # Agregar similitud
        results_df['similitud'] = cosine_sims[top_indices]
        
        end_time = time.time()
        print(f"Tiempo procesamiento: {end_time - start_time:.4f} segindos.")
        
        return results_df

# Motor Recomendacion

if __name__ == "__main__":
    
    # Rutas de arrchivos
    EMBEDDING_FILE = 'perfume_embeddings.npy'
    INDEX_FILE = 'perfume_index.csv'
    MODEL_NAME = 'all-MiniLM-L6-v2'
    
    try:
        # 1. Inicializar el motor
        engine = RecommenderEngine(MODEL_NAME, EMBEDDING_FILE, INDEX_FILE)

        # 2.  consultas 
        
        print("Consulta 1:")
        query_1 = "algo fresco y limpio para el verano en la oficina"
        recomendaciones_1 = engine.find_similar(query_1, top_n=3)
        print(f"Query: '{query_1}'")
        print(recomendaciones_1)
        print("\n")
        
        print("Consulta 2:")
        query_2 = "perfume dulce con vainilla y chocolate"
        recomendaciones_2 = engine.find_similar(query_2, top_n=3)
        print(f"Query: '{query_2}'")
        print(recomendaciones_2)
        print("\n")
        
        
        
        
        
        print("Consulta 3:")
        query_3 = "fragancia fresca y chispeante para el verano"
        recomendaciones_3 = engine.find_similar(query_3, top_n=3)
        print(f"Query: '{query_3}'")
        print(recomendaciones_3)



    except FileNotFoundError as e:
        print(e)
        print("Ejecutar 'generar_embeddings.py' Para crear archivos necesarios.")

Total perfumes cargados: 24063
Forma matriz embedida: (24063, 384)
Consulta 1:
Tiempo procesamiento: 0.2058 segindos.
Query: 'algo fresco y limpio para el verano en la oficina'
       index               Perfume                   Brand  similitud
19835  19835         coeur-d-ylang  comptoir-sud-pacifique   0.389928
7169    7169  le-chant-de-camargue     l-artisan-parfumeur   0.358992
13905  13905                 caoba            fueguia-1833   0.355485


Consulta 2:
Tiempo procesamiento: 0.0943 segindos.
Query: 'perfume dulce con vainilla y chocolate'
       index          Perfume                   Brand  similitud
10412  10412  amalia-gourmand            fueguia-1833   0.629525
9955    9955            layan         junaid-perfumes   0.572637
19835  19835    coeur-d-ylang  comptoir-sud-pacifique   0.569871


Consulta 3:
Tiempo procesamiento: 0.0623 segindos.
Query: 'fragancia fresca y chispeante para el verano'
       index                Perfume                   Brand  similitud
9783